In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
# import tushare as ts
import QUANTAXIS as QA
import talib as ta
import datetime


In [2]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [5]:
us_data = pd.read_csv('data/us_data.csv')
us_data.columns = ['date', 'high', 'low', 'open', 'close', 'volume', 'Adj Close', 'code']
us_data = us_data.set_index(['date', 'code'])
us_data = QA.QA_DataStruct_Stock_day(us_data)
us_data.data.head()

high        low       open      close      volume  \
date       code                                                           
2009-12-31 A     22.725323  22.124464  22.203148  22.224607   5224600.0   
           AABA  16.959999  16.770000  16.920000  16.780001   9515600.0   
           AAP   41.110001  40.480000  41.110001  40.480000    894500.0   
           AAPL  30.478571  30.080000  30.447144  30.104286  88102700.0   
           AAXN   4.430000   4.320000   4.400000   4.380000    400400.0   

                 Adj Close  
date       code             
2009-12-31 A     20.562288  
           AABA  16.780001  
           AAP   39.562489  
           AAPL  20.159719  
           AAXN   4.380000

In [36]:
def get_data(path):
    data = pd.read_csv(path)
    data.columns = ['date', 'high', 'low', 'open', 'close', 'volume', 'Adj Close', 'code']
    data = data.set_index(['date', 'code'])
    data = QA.QA_DataStruct_Stock_day(data)
    return data

In [8]:
code_list_by_sector = pd.read_excel('data/etf_pair_code.xlsx', dtype={'symbol':str})

In [9]:
code_list_by_sector.head()

,symbol,region,etf_symbol,sector
0,MSFT,US Equity,XLK,Hardware
1,AAPL,US Equity,XLK,Hardware
2,V,US Equity,XLK,Hardware
3,INTC,US Equity,XLK,Hardware
4,CSCO,US Equity,XLK,Hardware


In [13]:
sector_list = code_list_by_sector.sector.unique().tolist()
sector_list

['Hardware',
 'Internet',
 'China Internet',
 'Online Gaming',
 'Software/ Cloud Computing',
 'Cloud Computing',
 'Network/Security',
 'ePayment',
 'Semiconductor',
 'Medical',
 'Biotech/ Pharma',
 'Health Care Services',
 'Defense',
 'Consumer',
 'Financials',
 'Energy',
 'Others']

In [20]:
region_list = code_list_by_sector.region.unique().tolist()
region_list

['US Equity',
 'HK Equity',
 'HKD Curncy',
 'KS Equity',
 'JP Equity',
 'FP Equity',
 'TT Equity',
 'SS Equity',
 'GR Equity',
 'LN Equity',
 'NO Equity',
 'FH Equity',
 'CN Equity',
 'NA Equity',
 'ita']

In [25]:
# code_list_by_sector.query('sector=="%s"' %\
#                           (sector_list[0]))#.symbol.tolist()
# code_list_by_sector.query('sector=="%s" & region=="%s"' %\
#                           (sector_list[0], 'US Equity'))#.symbol.tolist()

In [22]:
def get_code_list_by_sector(code_list_df, sector, region='US Equity'):
    if region == 'all':
        df = code_list_df.query('sector=="%s"' % (sector))
    else:
        df = code_list_df.query('sector=="%s" & region=="%s"' %\
                          (sector, region))
    
    return df

In [23]:
us_hardware_df = get_code_list_by_sector(code_list_by_sector, 
                sector_list[0])

In [27]:
us_hardware_df.head()

,symbol,region,etf_symbol,sector
0,MSFT,US Equity,XLK,Hardware
1,AAPL,US Equity,XLK,Hardware
2,V,US Equity,XLK,Hardware
3,INTC,US Equity,XLK,Hardware
4,CSCO,US Equity,XLK,Hardware


In [28]:
us_data.select_code(us_hardware_df.symbol.tolist())

< QA_DataStruct_Stock_day with 68 securities >

In [29]:
lens = len(us_hardware_df.symbol.tolist())

In [30]:
from tqdm import tqdm_notebook

In [33]:
# for i in tqdm_notebook(range(lens)):
# #     print(i)

1. 将单pair做成一个函数
    - 先核对港股的pair是否正确
2. 然后再做循环
3. 做完这个再做买卖点

In [37]:
hk_data = get_data('data/hk_data.csv')

In [40]:
test_data = hk_data.select_code(['0939.hk','1398.hk'])

In [43]:
import pandas_datareader.data as web

In [58]:
code = ['0939.hk','1398.hk']
start_date = '2016-01-10'
end_date = '2018-12-04'
js_data = web.DataReader(code[0], 'yahoo', start_date, end_date)
gs_data = web.DataReader(code[1], 'yahoo', start_date, end_date)

In [59]:
js_close = js_data['Adj Close']
gs_close = gs_data['Adj Close']

In [67]:
js_close.head()

Date
2016-01-11    4.215010
2016-01-12    4.189308
2016-01-13    4.189308
2016-01-14    4.129339
2016-01-15    4.052235
Name: Adj Close, dtype: float64

In [68]:
gs_close.head()

Date
2016-01-11    3.566770
2016-01-12    3.516058
2016-01-13    3.524510
2016-01-14    3.507606
2016-01-15    3.423086
Name: Adj Close, dtype: float64

In [69]:
ratio = js_close / gs_close

In [70]:
ratio.head()

Date
2016-01-11    1.181744
2016-01-12    1.191479
2016-01-13    1.188621
2016-01-14    1.177253
2016-01-15    1.183796
Name: Adj Close, dtype: float64

In [71]:
position_side_list = []
for i in range(ratio.shape[0]):
    if i > 1:
        if ratio[i] > np.mean(ratio[:i-1]):
            position_side_list.append(1)
        elif ratio[i] < np.mean(ratio[:i-1]):
            position_side_list.append(-1)
        else:
            position_side_list.append(np.nan)
    elif i == 1:
        if ratio[1] > ratio[0]:
            position_side_list.append(1)
        elif ratio[1] < ratio[0]:
            position_side_list.append(-1)
        else:
            position_side_list.append(np.nan)
    elif i == 0:
        position_side_list.append(np.nan)
position_side_series = pd.Series(position_side_list, ratio.index)

In [72]:
backtest_position_side = position_side_series.shift(1)

In [73]:
backtest_position_side

Date
2016-01-11    NaN
2016-01-12    NaN
2016-01-13    1.0
2016-01-14    1.0
2016-01-15   -1.0
2016-01-18   -1.0
2016-01-19   -1.0
2016-01-20    1.0
2016-01-21   -1.0
2016-01-22   -1.0
2016-01-25   -1.0
2016-01-26   -1.0
2016-01-27   -1.0
2016-01-28    1.0
2016-01-29    1.0
2016-02-01    1.0
2016-02-02    1.0
2016-02-03    1.0
2016-02-04   -1.0
2016-02-05   -1.0
2016-02-11   -1.0
2016-02-12   -1.0
2016-02-15   -1.0
2016-02-16   -1.0
2016-02-17   -1.0
2016-02-18   -1.0
2016-02-19    1.0
2016-02-22    1.0
2016-02-23    1.0
2016-02-24    1.0
             ... 
2018-10-25   -1.0
2018-10-26   -1.0
2018-10-29   -1.0
2018-10-30   -1.0
2018-10-31   -1.0
2018-11-01   -1.0
2018-11-02   -1.0
2018-11-05   -1.0
2018-11-06   -1.0
2018-11-07   -1.0
2018-11-08   -1.0
2018-11-09   -1.0
2018-11-12    1.0
2018-11-13    1.0
2018-11-14    1.0
2018-11-15   -1.0
2018-11-16    1.0
2018-11-19    1.0
2018-11-20    1.0
2018-11-21   -1.0
2018-11-22   -1.0
2018-11-23   -1.0
2018-11-26   -1.0
2018-11-27   -1.0
2018-

In [74]:
js_pct = js_close.pct_change()
gs_pct = gs_close.pct_change()

In [75]:
((1+backtest_position_side)/2 * js_pct + \
 (1-backtest_position_side)/2 * gs_pct).mean()

0.0005036333250701651

In [78]:
(backtest_position_side * (js_pct - gs_pct)).mean()

-0.0005338697374752846